# HuggingTweets - Tweet Generation with Huggingface

*Disclaimer: this project is not to be used to publish any false generated information but to perform research on Natural Language Generation (NLG).*

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/borisdayma/huggingtweets/blob/master/huggingtweets.ipynb)

## Pre colab



1. Download csvfilter.py
2. Download the discord messages for the person as csv
3. merge the csv's together
4. run csvfilter.py and follow instructions
5. once it asks for content and number removal go to edit-csv and remove the first columm and row
6. when it asks for whitename removal execute the powershell script then press return on the script
7. the ouput will be the fully filtered csv.

## Install dependencies

In [1]:
# install required libraries are not installed
!pip install torch -qq
!pip install transformers==v3.4.0 -qq
!pip install wandb==v0.12.15 -qq

     |████████████████████████████████| 1.3 MB 5.0 MB/s 
     |████████████████████████████████| 895 kB 47.5 MB/s 
     |████████████████████████████████| 2.9 MB 42.5 MB/s 
     |████████████████████████████████| 1.2 MB 39.5 MB/s 
     |████████████████████████████████| 1.8 MB 5.1 MB/s 
     |████████████████████████████████| 181 kB 61.4 MB/s 
     |████████████████████████████████| 144 kB 61.9 MB/s 
     |████████████████████████████████| 63 kB 2.2 MB/s 


In [2]:
# HuggingFace scripts for fine-tuning models and language generation
!wget https://raw.githubusercontent.com/huggingface/transformers/v3.3.0/examples/language-modeling/run_language_modeling.py
!wget https://raw.githubusercontent.com/huggingface/transformers/v3.5.1/examples/text-generation/run_generation.py

--2022-04-23 20:22:44--  https://raw.githubusercontent.com/huggingface/transformers/v3.3.0/examples/language-modeling/run_language_modeling.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11434 (11K) [text/plain]
Saving to: ‘run_language_modeling.py’

run_language_modeli 100%[===================>]  11.17K  --.-KB/s    in 0s      

2022-04-23 20:22:44 (97.6 MB/s) - ‘run_language_modeling.py’ saved [11434/11434]

--2022-04-23 20:22:44--  https://raw.githubusercontent.com/huggingface/transformers/v3.5.1/examples/text-generation/run_generation.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... co

## Filter

filter the csv file


In [3]:
from array import array
import pandas as pd
import csv
import re

In [4]:
from google.colab import files
print("Use Py scripts to generate dataset csv")
files.upload()
!ls
csvanme = input("enter csv name")


Use Py scripts to generate dataset csv


Saving cleanNVG.csv to cleanNVG.csv
cleanNVG.csv  run_generation.py  run_language_modeling.py  sample_data
enter csv namecleanNVG.csv


## Make Dataset

help me please


In [5]:
import random
import re
import torch
import csv

In [6]:
handle = 'JustinHughes'
with open(csvanme, newline='') as csvfile:
    reader = csv.reader(csvfile, delimiter=' ', quotechar='|')
    reader = list(reader)
# shuffle data
    random.shuffle(reader)

# fraction of training data
    split_train_valid = 0.9

In [7]:
# split dataset
with open(csvanme, newline='') as csvfile:
    reader = csv.reader(csvfile, delimiter=' ', quotechar='|')
    reader = list(reader)
train_size = int(split_train_valid * len(reader))
valid_size = len(reader) - train_size
train_dataset, valid_dataset = torch.utils.data.random_split(reader, [train_size, valid_size]) 

def make_dataset(dataset, epochs):

  total_text = '<|endoftext|>'
  reader = [t for t in dataset]
  for _ in range(epochs):

    random.shuffle(reader)
    total_text += '<|endoftext|>'.join(map(str, reader)) + '<|endoftext|>'
  return total_text
EPOCHS = 4

with open('{}_train.txt'.format(handle), 'w') as f:

  data = make_dataset(train_dataset, EPOCHS)
  f.write(data)

with open('{}_valid.txt'.format(handle), 'w') as f:
  data = make_dataset(valid_dataset, 1)
  f.write(data)

## Log and monitor training through W&B

In order to check our model is training correctly and compare experiments, we are going to use the W&B integration from HuggingFace.

### API Key
Once you've signed up, run the next cell and click on the link to get your API key and authenticate this notebook.

In [10]:
import wandb
!wandb login --relogin

wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


## Fine-tuning the model

HuggingFace includes the script `run_language_modeling` making it easy to fine-tune a pre-trained model.

We use a pre-trained GPT-2 model and fine-tune it on our dataset.

Training is automatically logged on W&B (see [documentation](https://docs.wandb.com/huggingface)). Urls are generated to visualize ongoing runs or you can just open your [dashboard](http://app.wandb.ai/).

I quickly tested running for several epochs and my run was showing I started overfitting after 4 epochs so this is the limit I use to fine-tune my model (takes less than 2 minutes).

![](https://i.imgur.com/1uIxLFe.png)

In [11]:
# Associate run to a project (optional)
%env WANDB_PROJECT=huggingtweets-dev

env: WANDB_PROJECT=huggingtweets-dev


We use HuggingFace script `run_language_modeling.py` to fine-tune our model (see [doc](https://huggingface.co/transformers/)).

*Note: epochs are built into the dataset*

In [12]:
!python run_language_modeling.py \
    --output_dir=output/$handle \
    --overwrite_output_dir \
    --overwrite_cache \
    --model_type=gpt2 \
    --model_name_or_path=gpt2 \
    --do_train --train_data_file=$handle\_train.txt \
    --do_eval --eval_data_file=$handle\_valid.txt \
    --eval_steps 20 \
    --logging_steps 20 \
    --per_gpu_train_batch_size 1 \
    --num_train_epochs 1

04/23/2022 20:24:16 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 1, distributed training: False, 16-bits training: False
04/23/2022 20:24:16 - INFO - __main__ -   Training/evaluation parameters TrainingArguments(output_dir='output/JustinHughes', overwrite_output_dir=True, do_train=True, do_eval=True, do_predict=False, evaluate_during_training=False, evaluation_strategy=<EvaluationStrategy.NO: 'no'>, prediction_loss_only=False, per_device_train_batch_size=8, per_device_eval_batch_size=8, per_gpu_train_batch_size=1, per_gpu_eval_batch_size=None, gradient_accumulation_steps=1, eval_accumulation_steps=None, learning_rate=5e-05, weight_decay=0.0, adam_beta1=0.9, adam_beta2=0.999, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=1.0, max_steps=-1, warmup_steps=0, logging_dir='runs/Apr23_20-24-16_34b79911ee29', logging_first_step=False, logging_steps=20, save_steps=500, save_total_limit=None, no_cuda=False, seed=42, fp16=False, fp16_opt_level='O1', local_rank=-1, t

## Let's test our trained model!

We test our model on a few sample sentences.

In [13]:
SENTENCES = ["I think that",
             "I like",
             "I don't like",
             "I want",
             "My dream is"]

We use HuggingFace script `run_generation.py` to generate sentences (see [doc](https://huggingface.co/transformers/)).

In [14]:
import random
seed = random.randint(0, 2**32-1)
seed

1086660266

In [ ]:
!

In [15]:
examples = []
num_return_sequences = 5

for start in SENTENCES:
    val = !python run_generation.py \
        --model_type gpt2 \
        --model_name_or_path output/$handle \
        --length 160 \
        --num_return_sequences $num_return_sequences \
        --temperature 1 \
        --p 0.95 \
        --seed $seed \
        --prompt {'"<|endoftext|>' + start + '"'}
    generated = [val[-1-2*k] for k in range(num_return_sequences)[::-1]]
    print(f'\nStart of sentence: {start}')
    for i, g in enumerate(generated):
        g = g.replace('<|endoftext|>', '')
        print(f'* Generated #{i+1}: {g}')


Start of sentence: I think that
* Generated #1: === GENERATED SEQUENCE 3 ===
* Generated #2: 
* Generated #3: Just in case everyone else thought I might've been a little misguided, thanks!
* Generated #4: I think that's cool, but actually until someone takes the plunge I'm not sure how long they're gonna wait, which i'm guessing is until you're shit." -@ryginthenothing
* Generated #5: I think that the commander-in-chief needs to find a better way to recruit the soldiers, particularly in a modern-day Iraq where sectarian tensions between Sunni tribes are even wider, and if the rules of engagement are not enforced, they'll emerge as a threat to the brand-new constitution.

Start of sentence: I like
* Generated #1: I like fun! It's just until someone gets frustrated, laughs, makes fun of you...eyersomieyeU! If you've gotten much shitim laughing, come on! I'm super sorry."
* Generated #2: "You're right, too."
* Generated #3: "You're"
* Generated #4: I like the portraits! And the majority 

## About

*Built by Boris Dayma* *Modified by UN! and Amphe Co.*

[![Follow](https://img.shields.io/twitter/follow/borisdayma?style=social)](https://twitter.com/intent/follow?screen_name=borisdayma)

[![Follow](https://img.shields.io/twitter/follow/lilunsimp?style=social)](https://twitter.com/intent/follow?screen_name=lilunsimp)

[![Follow](https://img.shields.io/twitter/follow/ampheos?style=social)](https://twitter.com/intent/follow?screen_name=ampheos)

[![GitHub followers](https://img.shields.io/github/followers/un-simp?label=un-simp&style=social)](https://github.com/un-simp)

[![GitHub Org's stars](https://img.shields.io/github/stars/amphe-os?label=amphe-OS&style=social)](https://github.com/amphe-os)

My main goals with this project are:
* to experiment with how to train, deploy and maintain neural networks in production ;
* to make AI accessible to everyone ;
* to have fun!

For more details, visit the project repository.

[![GitHub stars](https://img.shields.io/github/stars/borisdayma/huggingtweets?style=social)](https://github.com/borisdayma/huggingtweets)

**Disclaimer: this project is not to be used to publish any false generated information but to perform research on Natural Language Generation.**

## Resources

* [Explore the W&B report](https://app.wandb.ai/wandb/huggingtweets/reports/HuggingTweets-Train-a-model-to-generate-tweets--VmlldzoxMTY5MjI) to understand how the model works
* [HuggingFace and W&B integration documentation](https://docs.wandb.com/library/integrations/huggingface)

## Got questions about W&B?

If you have any questions about using W&B to track your model performance and predictions, please reach out to the [slack community](http://bit.ly/wandb-forum).